## Step 1: Install Dependencies

Make sure MediaPipe is installed.

In [28]:
!pip install -q mediapipe opencv-python

## Step 2: Download the Model

Download the hand landmarker model if it doesn't exist already.

In [29]:
import os
import urllib.request

# Only download if the model doesn't exist
if not os.path.exists('hand_landmarker.task'):
    print("Downloading model...")
    urllib.request.urlretrieve(
        'https://storage.googleapis.com/mediapipe-models/hand_landmarker/hand_landmarker/float16/1/hand_landmarker.task',
        'hand_landmarker.task'
    )
    print("Model downloaded successfully!")
else:
    print("Model already exists!")

Model already exists!


## Step 3: Import Libraries

Import all necessary libraries for video capture and hand detection.

In [30]:
import cv2
import mediapipe as mp
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import numpy as np
import time

print("All libraries imported successfully!")

All libraries imported successfully!


## Step 4: Visualization Function

This function draws the hand landmarks and skeleton on the image.

**What it does:**
- Loops through each detected hand
- Draws 21 landmark points connected by lines
- Labels each hand as "Left" or "Right"

In [ ]:
# Constants for drawing
MARGIN = 10  # pixels above hand to place label
FONT_SIZE = 1
FONT_THICKNESS = 2
HANDEDNESS_TEXT_COLOR = (88, 205, 54)  # Vibrant green (BGR format)

# Hand landmark indices
WRIST = 0
THUMB_CMC = 1
THUMB_MCP = 2
THUMB_IP = 3
THUMB_TIP = 4
INDEX_MCP = 5
INDEX_TIP = 8
MIDDLE_MCP = 9
MIDDLE_TIP = 12
RING_MCP = 13
RING_TIP = 16
PINKY_MCP = 17
PINKY_TIP = 20

def detect_thumbs_gesture(hand_landmarks):
    """
    Detects thumbs up or thumbs down gesture.
    
    Logic:
    - Thumbs UP: Thumb tip is ABOVE (lower y) the thumb MCP, and other fingers are curled
    - Thumbs DOWN: Thumb tip is BELOW (higher y) the thumb MCP, and other fingers are curled
    - Fingers are "curled" when their tips are below their MCP joints (higher y value)
    
    Args:
        hand_landmarks: List of 21 hand landmarks from MediaPipe
    
    Returns:
        "thumbs_up", "thumbs_down", or None
    """
    # Get relevant landmarks
    thumb_tip = hand_landmarks[THUMB_TIP]
    thumb_mcp = hand_landmarks[THUMB_MCP]
    thumb_ip = hand_landmarks[THUMB_IP]
    wrist = hand_landmarks[WRIST]
    
    index_tip = hand_landmarks[INDEX_TIP]
    index_mcp = hand_landmarks[INDEX_MCP]
    middle_tip = hand_landmarks[MIDDLE_TIP]
    middle_mcp = hand_landmarks[MIDDLE_MCP]
    ring_tip = hand_landmarks[RING_TIP]
    ring_mcp = hand_landmarks[RING_MCP]
    pinky_tip = hand_landmarks[PINKY_TIP]
    pinky_mcp = hand_landmarks[PINKY_MCP]
    
    # Check if fingers are curled (tips below MCPs - higher y value means lower on screen)
    index_curled = index_tip.y > index_mcp.y
    middle_curled = middle_tip.y > middle_mcp.y
    ring_curled = ring_tip.y > ring_mcp.y
    pinky_curled = pinky_tip.y > pinky_mcp.y
    
    # All four fingers must be curled for a thumbs gesture
    fingers_curled = index_curled and middle_curled and ring_curled and pinky_curled
    
    if not fingers_curled:
        return None
    
    # Calculate thumb direction
    # Thumb tip vs thumb MCP (base of thumb)
    thumb_vertical_diff = thumb_mcp.y - thumb_tip.y  # Positive = thumb pointing up
    
    # Threshold for detection (normalized coordinates, so ~0.1 is significant)
    threshold = 0.07
    
    # Also check thumb is extended (tip is far from IP joint)
    thumb_extended = abs(thumb_tip.y - thumb_ip.y) > 0.02
    
    if thumb_vertical_diff > threshold and thumb_extended:
        return "thumbs_up"
    elif thumb_vertical_diff < -threshold and thumb_extended:
        return "thumbs_down"
    
    return None

def draw_landmarks_on_image(rgb_image, detection_result):
    """
    Draws hand landmarks, handedness labels, and gesture detection on an image.
    
    Args:
        rgb_image: Input image as NumPy array (RGB format)
        detection_result: MediaPipe HandLandmarkerResult object
    
    Returns:
        Tuple of (annotated_image, list of detected gestures)
    """
    # Get lists of landmarks and handedness from detection result
    hand_landmarks_list = detection_result.hand_landmarks
    handedness_list = detection_result.handedness
    
    # Create a copy so we don't modify the original
    annotated_image = np.copy(rgb_image)
    
    # Store detected gestures
    gestures = []

    # Loop through each detected hand
    for idx in range(len(hand_landmarks_list)):
        hand_landmarks = hand_landmarks_list[idx]  # 21 landmarks for this hand
        handedness = handedness_list[idx]  # Left or Right classification

        # Detect thumbs gesture
        gesture = detect_thumbs_gesture(hand_landmarks)
        
        # Get corrected handedness (for mirrored display)
        original_label = handedness[0].category_name
        corrected_label = "Right" if original_label == "Left" else "Left"
        
        gestures.append({
            "hand": corrected_label,
            "gesture": gesture
        })

        # Convert landmarks to protobuf format (required by MediaPipe drawing utils)
        hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
        hand_landmarks_proto.landmark.extend([
            landmark_pb2.NormalizedLandmark(
                x=landmark.x, 
                y=landmark.y, 
                z=landmark.z
            ) for landmark in hand_landmarks
        ])
        
        # Draw the hand skeleton (dots + connecting lines)
        solutions.drawing_utils.draw_landmarks(
            annotated_image,
            hand_landmarks_proto,
            solutions.hands.HAND_CONNECTIONS,  # Defines which landmarks to connect
            solutions.drawing_styles.get_default_hand_landmarks_style(),
            solutions.drawing_styles.get_default_hand_connections_style()
        )

        # Calculate position for the handedness label (top-left of hand)
        height, width, _ = annotated_image.shape
        x_coordinates = [landmark.x for landmark in hand_landmarks]
        y_coordinates = [landmark.y for landmark in hand_landmarks]
        
        # Convert normalized coordinates (0-1) to pixel coordinates
        text_x = int(min(x_coordinates) * width)
        text_y = int(min(y_coordinates) * height) - MARGIN
        
        # Build label with gesture
        if gesture == "thumbs_up":
            label = f"{corrected_label} 👍 THUMBS UP!"
            color = (0, 255, 0)  # Green for thumbs up
        elif gesture == "thumbs_down":
            label = f"{corrected_label} 👎 THUMBS DOWN!"
            color = (255, 0, 0)  # Red for thumbs down
        else:
            label = corrected_label
            color = HANDEDNESS_TEXT_COLOR
        
        cv2.putText(
            annotated_image,
            label,
            (text_x, text_y),
            cv2.FONT_HERSHEY_DUPLEX,
            FONT_SIZE,
            color,
            FONT_THICKNESS,
            cv2.LINE_AA  # Anti-aliased (smooth) text
        )

    return annotated_image, gestures

print("Visualization and gesture detection functions defined!")
print("Gestures detected: Thumbs Up 👍, Thumbs Down 👎")

Visualization and gesture detection functions defined!
Gestures detected: Thumbs Up 👍, Thumbs Down 👎


## Step 5: Initialize the Hand Detector

Create the HandLandmarker detector configured for VIDEO mode.

**VIDEO mode vs IMAGE mode:**
- VIDEO mode uses tracking between frames (faster, smoother)
- Requires a timestamp for each frame

In [32]:
# Configure the detector
base_options = python.BaseOptions(model_asset_path='hand_landmarker.task')

options = vision.HandLandmarkerOptions(
    base_options=base_options,
    running_mode=vision.RunningMode.VIDEO,  # Optimized for video frames
    num_hands=2,  # Detect up to 2 hands
    min_hand_detection_confidence=0.5,  # Minimum confidence to detect a hand
    min_hand_presence_confidence=0.5,   # Minimum confidence hand is still present
    min_tracking_confidence=0.5         # Minimum confidence for tracking between frames
)

# Create the detector
detector = vision.HandLandmarker.create_from_options(options)

print("Hand detector initialized!")
print(f"- Running mode: VIDEO")
print(f"- Max hands: 2")
print(f"- Detection confidence: 0.5")

Hand detector initialized!
- Running mode: VIDEO
- Max hands: 2
- Detection confidence: 0.5


I0000 00:00:1768382607.221729    5709 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1768382607.223716    6421 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 25.0.7-0ubuntu0.24.04.2), renderer: Mesa Intel(R) Graphics (ADL GT2)
W0000 00:00:1768382607.235733    6432 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1768382607.248716    6425 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


## Step 6: Run Live Detection with Gesture Recognition! 🎥

This cell opens your webcam and runs hand detection with **thumbs up/down gesture recognition**.

**Gesture Detection Logic:**
- **Thumbs UP 👍**: Thumb tip is ABOVE the thumb base, all other fingers are curled (closed fist)
- **Thumbs DOWN 👎**: Thumb tip is BELOW the thumb base, all other fingers are curled

**Controls:**
- Press **'q'** to quit and close the window
- Press **'b'** to toggle Blue Glove Mode

**What you'll see:**
- Live video feed with hand landmarks drawn
- FPS (frames per second) displayed
- "Left" or "Right" labels on each detected hand
- **Green "THUMBS UP!"** when showing thumbs up
- **Red "THUMBS DOWN!"** when showing thumbs down

In [ ]:
# Open the webcam (0 = default camera, try 1 or 2 if you have multiple cameras)
cap = cv2.VideoCapture(0)

# Check if camera opened successfully
if not cap.isOpened():
    print("ERROR: Could not open camera!")
    print("Try changing VideoCapture(0) to VideoCapture(1)")
else:
    print("Camera opened successfully!")
    print("Press 'q' to quit, 'b' to toggle Blue Glove Mode")
    print("-" * 40)
    print("Show THUMBS UP 👍 or THUMBS DOWN 👎 to the camera!")
    print("-" * 40)

# Variables for FPS calculation
prev_time = 0
fps = 0

# Toggle for blue glove mode (swap R and B channels)
BLUE_GLOVE_MODE = True  # Set to True when wearing blue gloves

# Gesture counters for display
gesture_counts = {"thumbs_up": 0, "thumbs_down": 0}

# Main loop - runs until 'q' is pressed
while cap.isOpened():
    # Step 1: Read a frame from the webcam
    success, frame = cap.read()
    
    if not success:
        print("Failed to read frame from camera")
        break
    
    # Step 2: Flip the frame horizontally (mirror effect - more intuitive)
    frame = cv2.flip(frame, 1)
    
    # Step 3: Convert BGR (OpenCV format) to RGB (MediaPipe format)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Step 3.5: BLUE GLOVE MODE - Swap R and B channels for detection
    if BLUE_GLOVE_MODE:
        frame_for_detection = np.ascontiguousarray(frame_rgb[:, :, [2, 1, 0]])
    else:
        frame_for_detection = frame_rgb
    
    # Step 4: Create MediaPipe Image object
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame_for_detection)
    
    # Step 5: Get current timestamp in milliseconds
    timestamp_ms = int(time.time() * 1000)
    
    # Step 6: Run hand detection
    detection_result = detector.detect_for_video(mp_image, timestamp_ms)
    
    # Step 7: Draw landmarks and detect gestures
    annotated_frame, gestures = draw_landmarks_on_image(frame_rgb, detection_result)
    
    # Count gestures
    for g in gestures:
        if g["gesture"] == "thumbs_up":
            gesture_counts["thumbs_up"] += 1
        elif g["gesture"] == "thumbs_down":
            gesture_counts["thumbs_down"] += 1
    
    # Step 8: Convert back to BGR for OpenCV display
    annotated_frame_bgr = cv2.cvtColor(annotated_frame, cv2.COLOR_RGB2BGR)
    
    # Step 9: Calculate and display FPS
    current_time = time.time()
    fps = 1 / (current_time - prev_time) if prev_time > 0 else 0
    prev_time = current_time
    
    # Draw FPS on the frame
    cv2.putText(
        annotated_frame_bgr,
        f"FPS: {fps:.1f}",
        (10, 30),
        cv2.FONT_HERSHEY_SIMPLEX,
        1,
        (0, 255, 0),
        2,
        cv2.LINE_AA
    )
    
    # Draw number of hands detected
    num_hands = len(detection_result.hand_landmarks)
    cv2.putText(
        annotated_frame_bgr,
        f"Hands: {num_hands}",
        (10, 70),
        cv2.FONT_HERSHEY_SIMPLEX,
        1,
        (0, 255, 0),
        2,
        cv2.LINE_AA
    )
    
    # Show blue glove mode status
    mode_text = "Blue Glove: ON" if BLUE_GLOVE_MODE else "Blue Glove: OFF"
    cv2.putText(
        annotated_frame_bgr,
        mode_text,
        (10, 110),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.7,
        (255, 165, 0) if BLUE_GLOVE_MODE else (128, 128, 128),
        2,
        cv2.LINE_AA
    )
    
    # Show gesture instructions
    cv2.putText(
        annotated_frame_bgr,
        "Show THUMBS UP or DOWN!",
        (10, 150),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.7,
        (255, 255, 255),
        2,
        cv2.LINE_AA
    )
    
    # Display current gesture detection in big text at bottom
    current_gesture = None
    for g in gestures:
        if g["gesture"]:
            current_gesture = g["gesture"]
            break
    
    if current_gesture == "thumbs_up":
        cv2.putText(
            annotated_frame_bgr,
            "THUMBS UP!",
            (frame.shape[1] // 2 - 150, frame.shape[0] - 50),
            cv2.FONT_HERSHEY_SIMPLEX,
            1.5,
            (0, 255, 0),
            3,
            cv2.LINE_AA
        )
    elif current_gesture == "thumbs_down":
        cv2.putText(
            annotated_frame_bgr,
            "THUMBS DOWN!",
            (frame.shape[1] // 2 - 180, frame.shape[0] - 50),
            cv2.FONT_HERSHEY_SIMPLEX,
            1.5,
            (0, 0, 255),
            3,
            cv2.LINE_AA
        )
    
    # Step 10: Display the frame in a window
    cv2.imshow('Hand Gesture Detection - Press Q to Quit, B to Toggle Blue Glove Mode', annotated_frame_bgr)
    
    # Step 11: Check for key presses
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        print("\nQuitting...")
        break
    elif key == ord('b'):
        BLUE_GLOVE_MODE = not BLUE_GLOVE_MODE
        status = "ON" if BLUE_GLOVE_MODE else "OFF"
        print(f"Blue Glove Mode: {status}")

# Cleanup: Release the camera and close windows
cap.release()
cv2.destroyAllWindows()

print("Camera released and windows closed!")
print(f"\nGesture Summary:")
print(f"  Thumbs Up detected: {gesture_counts['thumbs_up']} frames")
print(f"  Thumbs Down detected: {gesture_counts['thumbs_down']} frames")

Camera opened successfully!
Press 'q' to quit, 'b' to toggle Blue Glove Mode
----------------------------------------
Show THUMBS UP 👍 or THUMBS DOWN 👎 to the camera!
----------------------------------------
Blue Glove Mode: OFF
Blue Glove Mode: ON


## Troubleshooting

| Problem | Solution |
|---------|----------|
| Camera not found | Change `VideoCapture(0)` to `VideoCapture(1)` or `VideoCapture(2)` |
| Window doesn't close | Make sure to press 'q' key, not the X button |
| Very slow/laggy | Reduce frame size or check CPU/GPU usage |
| Model not found | Run the model download cell first |
| Hands not detected | Ensure good lighting and hands are clearly visible |

## Optional: Reduce Frame Size for Better Performance

If the detection is slow, you can reduce the frame resolution.

In [ ]:
# Run this cell if you want to use a lower resolution for better performance
# Then run the main detection cell again

cap = cv2.VideoCapture(0)

# Set resolution to 640x480 (lower = faster)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

print(f"Resolution set to: {int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))}x{int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))}")

cap.release()

Resolution set to: 640x480
